In [5]:
# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
import joblib

# 1️⃣ Load dataset
data = pd.read_csv("../data/diabetes.csv")
X = data.drop("Outcome", axis=1)
y = data["Outcome"]

# 2️⃣ Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3️⃣ Handle class imbalance
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# 4️⃣ Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

# 5️⃣ Train Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"
)
rf_model.fit(X_train_scaled, y_train_res)

# 6️⃣ Evaluate
y_pred = rf_model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# 7️⃣ Save pipeline (scaler + model)
pipeline = Pipeline([
    ("scaler", scaler),
    ("model", rf_model)
])
joblib.dump(pipeline, "../models/diabeties_model_rf.pkl")

# ✅ Test single prediction
X_new = pd.DataFrame([[2, 120, 70, 20, 85, 25.3, 0.351, 30]],
                     columns=X.columns)
y_new = pipeline.predict(X_new)
proba_new = pipeline.predict_proba(X_new)
print("Predicted class:", y_new[0])
print(f"Probability of diabetes: {proba_new[0,1]:.2%}")


Accuracy: 0.7662337662337663
              precision    recall  f1-score   support

           0       0.85      0.78      0.81       100
           1       0.65      0.74      0.69        54

    accuracy                           0.77       154
   macro avg       0.75      0.76      0.75       154
weighted avg       0.78      0.77      0.77       154

Predicted class: 0
Probability of diabetes: 20.50%
